In [1]:
import syllables 
import pandas as pd
import random
import numpy as np
import re
import language_tool_python
from IPython.display import display
import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import timeit

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jennifergutman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import language_tool_python

In [3]:
tool = language_tool_python.LanguageTool('en-US')

In [4]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jennifergutman/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [5]:
# !pip install gingerit

In [6]:
with open("Dataset_poems.txt", "r",encoding="utf8") as file:
    # Trial_data.txt
    # Dataset_poems.txt
    temp = []
    for line in file:
        line1 = re.sub('[^a-zA-Z ]', '', line)
        line1 = line1.split()
        if line1 != ['', ''] and line1 != ['']:
            for i in line1:
                if i == '':
                    continue
                else:
                    temp.append(i.lower())
words_df = temp[:5000] 

In [7]:
tokens_tag = pos_tag(words_df)
dic_tag = {}
for tup in tokens_tag:
  if tup[1] not in dic_tag.keys():
    dic_tag[tup[1]] = [tup[0]]
  else:
    temp = dic_tag[tup[1]] + [tup[0]]
    dic_tag[tup[1]] = temp
# print(dic_tag.keys())
z = list(dic_tag.values())
words_df2 = []
for i in range(len(z)):
  words_df2.extend(z[i])
# print(words_df2)
words_df = words_df2
words_df.remove('o')
words_df.remove('n')

In [8]:
lines = []
for i in range(3, len(words_df)-3):
    lines.append(words_df[i-3]+' '+' '+words_df[i-2]+' '+' '+words_df[i-1]+' '+' '+words_df[i]+' '+' '+words_df[i+1]+' '+' '+words_df[i+2]+' '+' '+words_df[i+3])

# sklearn countvectorizer
from sklearn.feature_extraction.text import CountVectorizer
# Convert a collection of text documents to a matrix of token counts
cv = CountVectorizer(ngram_range=(1,1), stop_words = stopwords.words('english'))
# matrix of token counts
X = cv.fit_transform(lines)
Xc = (X.T * X) # matrix manipulation
Xc.setdiag(0) # set the diagonals to be zeroes as it's pointless to be 1

names = cv.get_feature_names() # This are the entity names (i.e. keywords)
cooccur_df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
total = cooccur_df.to_numpy().sum()
cooccur_df = cooccur_df.div(total)
cooccur_df.to_csv('co_occur_mat.csv', sep = ',')

In [9]:
class poem_generation:
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', None)
    pd.set_option('display.max_colwidth', None)

    def select_random_words(self,words_df):
        words_array1 = []
        while len(words_array1) <= 15:
            random_number = random.randint(0, len(words_df)-1)
            if words_df[random_number] != 0:
                words_array1.append(words_df[random_number])
        # print(len(words_array1))

        return words_array1

    def create_poems(self,arr):
        poem = [0 for _ in range(4)]
        for i in range(1,5):
            temp =  arr[4*(i-1):4*i]
            poem[i-1] = temp
            # print("HERE:",temp)
            # print(type(temp))
            poem[i-1]=' '.join(poem[i-1])
            # print(poem[i-1])
        # print("=========================================================")
        return poem

    def store_syllables(self,arr):
        a = [0 for _ in range(4)]
        temp = 0
        j = 0
        for words,i in zip(arr,range(0,16)):
            temp = temp + syllables.estimate(words)
            if i in [3,7,11,15]:
                a[j] = temp
                # print(a[j])
                temp = 0
                j += 1
        return a

    def Grammar_checker(self,text):
        error = 0
        poem_error = []
        # tool = language_tool_python.LanguageTool('en-US')
        for lines in text:
            matches = tool.check(lines)
            if len(matches)>0:
                error = error + matches[0].errorLength
        poem_error.append(error)
        return poem_error

    def crossover(self,p1,p2,cross):
        part1 = []
        part2 = []
        part3 = []
        child1 = []
        child2 = []
        j = 0
        k = 0
        random_number1 = sorted([random.randint(1, 15) for i in range(cross)])
        random_number2 = sorted([random.randint(1, 15) for i in range(cross)])
        for i in random_number1:
            part1.append(p1[j:i])
            j = i
        for i in random_number2:
            part2.append(p2[k:i])
            k = i
        part1.append(p1[j:])
        part2.append(p2[k:])
        part2.extend(part1)
        random.shuffle(part2)
        for i in range(len(part2)):
            part3.extend(part2[i])
        for i in range(int(len(part3)/2)):
            child1.append(part3[i])
        for i in range(int(len(part3)/2),int(len(part3))):
            child2.append(part3[i])
        # if child1 == p1 or child1 == p2 or child2 == p1 or child2 == p2:
        #     child1, child2,child1_poem,child2_poem = self.crossover(child1,child2,cross)
        child1_poem = self.create_poems(child1)
        child2_poem = self.create_poems(child2)
        return child1,child2,child1_poem,child2_poem
    
    def coherence_checker(self, text):
        clean_text = ''
        for i in text:
            clean_text = clean_text + ' ' + i
        
        text_tokens = word_tokenize(clean_text)
        tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]
        
        coherence_val = 0
        pairs = 0
        for i in range(len(tokens_without_sw) - 1):
            for j in range(i+1, len(tokens_without_sw)):
                coherence_val += cooccur_df[tokens_without_sw[i]][tokens_without_sw[j]]
                pairs += 1
        
        if (pairs > 0):
            coherence_val /= pairs
        return [coherence_val]

    def tournament(self,Population,m,best):
        from_best = len(best)
        # print("best:",best)
        except_best = m - from_best
        # df = pd.DataFrame()
        #selected_polulation = pd.DataFrame(columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","word_array"])
        ######################### when no syllables rule
        # selected_polulation = Population.iloc[:nn]
        # random_number = random.sample(list(Population.iloc[nn:].index),p)
        # Population.sort_values(by=['GrammarError'], inplace=True)
        # Population2 = Population[~Population.isin(selected_polulation).all(1)]
        ######################### when syllables rule
        if len(best) >0:
            selected_polulation = []
            words_array_selected = []
            for b in best:
                c = Population.iloc[b].values[:-1]
                d = Population.iloc[b].values[-1]
                selected_polulation.append(c)
                words_array_selected.append(d)
            words_array_selected = pd.DataFrame(dict(zip(["words_array"],[words_array_selected])))
            selected_polulation = pd.DataFrame(selected_polulation,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
            selected_polulation = pd.concat([selected_polulation,words_array_selected],axis=1)
            # print("poems selected by user")
            # display(selected_polulation)
            Population2 = Population[~Population.isin(selected_polulation).all(1)]
            Population2 = Population2.sort_values(by=['GrammarError'])
            selected_polulation2 = Population2[(Population2["Syllables1"] == "5")&(Population2["Syllables2"] == "6")&(Population2["Syllables3"] == "6")&(Population2["Syllables4"] == "5")]
            # selected_polulation2 = Population2[(Population2["Syllables1"] >= "3")]
            # print("grammar sorted poems by syll")
            # display(selected_polulation2)
            from_sy = len(selected_polulation2.index)
            if from_sy >= int(except_best/2):
                from_sy = int(except_best/2)
            else:
                from_sy = from_sy
            from_random = except_best - from_sy
            selected_polulation2 = selected_polulation2.iloc[:from_sy]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            display(selected_polulation)
            display(Population2)
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            Population2 = Population2.reset_index(drop=True)
            selected_polulation = pd.concat([selected_polulation,Population2.iloc[:from_random]],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected
        else:
            Population2 = Population.sort_values(by=['GrammarError'])
            random = int(except_best/3)
            grammar = except_best - random
            selected_polulation = Population2[:grammar]
            Population2 = Population2[~Population2.isin(selected_polulation).all(1)]
            selected_polulation2 = Population2[:random]
            selected_polulation = pd.concat([selected_polulation,selected_polulation2],axis=0)
            selected_polulation = selected_polulation.reset_index(drop=True)
            # print("SELECTED")
            # display(selected_polulation)
            words_array_selected = list(selected_polulation["words_array"])
            # print("TYPE",type(selected_polulation["words_array"]),words_array_selected)
            return selected_polulation,words_array_selected



    def create_set_of_poems(self,n):
            poem_population = []
            syllables_count = []
            grammar_error = []
            words_array_population = []
            coherence_arr = []
            for i in range(n):
                words_array = self.select_random_words(words_df)
                poem = self.create_poems(words_array)
                sum_syllables = self.store_syllables(words_array)
                poem_error = self.Grammar_checker(poem)
                coherence = self.coherence_checker(poem)
                poem_population.append(poem)
                syllables_count.append(sum_syllables)
                grammar_error.append(poem_error)
                coherence_arr.append(coherence)
                words_array_population.append(words_array)
            words_array2 = pd.DataFrame(dict(zip(["words_array"],[words_array_population])))
            Final = np.hstack((poem_population,syllables_count,grammar_error,coherence_arr))
            Population_data = pd.DataFrame(Final,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
            Pop_data = pd.concat([Population_data,words_array2],axis=1)
            # print()
            # print("PARENTS")
            # display(Pop_data)

            return Pop_data

    def child_poems(self,words_array,cross):
        child_population = []
        syllables_count = []
        grammer_error = []
        words_array_child = []
        coherence_arr = []
        # print("words_array",words_array)
        # print(type(words_array))
        if len(words_array)%2 == 2:
            for i in range(0,len(words_array)-1):
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                coherence1 = self.coherence_checker(child1_poem)
                coherence2 = self.coherence_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                coherence_arr.append(coherence1)
                coherence_arr.append(coherence2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        else:
            for i in range(0,len(words_array)-1,2):
                # print(words_array[i],words_array[i+1])
                child1,child2,child1_poem,child2_poem = self.crossover(words_array[i],words_array[i+1],cross)
                child_population.append(child1_poem)
                child_population.append(child2_poem)
                sum_syllables1 = self.store_syllables(child1)
                sum_syllables2 = self.store_syllables(child2)
                poem_error1 = self.Grammar_checker(child1_poem)
                poem_error2 = self.Grammar_checker(child2_poem)
                coherence1 = self.coherence_checker(child1_poem)
                coherence2 = self.coherence_checker(child2_poem)
                syllables_count.append(sum_syllables1)
                syllables_count.append(sum_syllables2)
                grammer_error.append(poem_error1)
                grammer_error.append(poem_error2)
                coherence_arr.append(coherence1)
                coherence_arr.append(coherence2)
                words_array_child.append(child1)
                words_array_child.append(child2)
        words_array_child = pd.DataFrame(dict(zip(["words_array"],[words_array_child])))
        Final1 = np.hstack((child_population,syllables_count,grammer_error,coherence_arr))
        child_data = pd.DataFrame(Final1,columns=["Line1","Line2","Line3","Line4","Syllables1","Syllables2","Syllables3","Syllables4","GrammarError","Coherence"])
        child_data = pd.concat([child_data,words_array_child],axis=1)
        child_data = child_data.reset_index(drop=True)
        print()
        # print("CHILD")
        # display(child_data)
        return child_data

    def selection_from_child_parents(self,x,child_data,selected_Pop_data):
        reduce = (len(list(child_data.index)) + len(list(selected_Pop_data.index))) - x
        n = int(reduce/3)
        p = reduce - n
        if reduce > len(list(child_data.index)) + len(list(selected_Pop_data.index)):
            print("ERROR")
        else:
            if n > len(child_data):
                n = len(child_data)
                p = reduce - p
            if n == 0:
                n = 1
                p = 1
        d = pd.concat([selected_Pop_data,child_data],axis=0)
        d = d.reset_index(drop=True)
        # print("p:",p)
        # print("n:",n)
        random_number1 = random.sample(list(selected_Pop_data.index),p)
        random_number2 = random.sample(list(range(len(list(selected_Pop_data.index)),len(list(d.index)))),n)
        random_number1.extend(random_number2)
        # print(random_number1)
        for i in random_number1:
          d.drop(i,axis=0,inplace = True)
        d = d.reset_index(drop=True)
        # print("NEW POPULATION")
        # display(d)
        r = d[["Line1","Line2","Line3","Line4"]]
        return d,r
    
    def __init__(self,words_df,n,cross,epoch,z):
        Selection_number = int(round(0.85*n))
        best = []
        assert n>1 ,f"Population {n} cannot be less than or equal to 1"
        # assert n>=Selection_number, f"Cannot select {Selection_number} poems from total population of {n}"
        assert cross>1, f"Minimum number for crossover should be 1"
        # assert 2*n >= Selection_number ,f"Cannot make initial population of {n} if only {Selection_number} parents are selected"

        Population_data = self.create_set_of_poems(n)
        for i in range(epoch):
            Selected_Population_data,selected_polulation_word_array = self.tournament(Population_data,Selection_number,best)
            child_population = self.child_poems(selected_polulation_word_array,cross)
            final_population,final_poems = self.selection_from_child_parents(n,child_population,Population_data)
            if i != epoch-1:
                if (i+1)%10 == 0: # User feedback every 10 epochs
                    print("Select best poems")
                    display(final_poems)
                    best = (list(map(int,input("\n Enter best poems numbers: ").strip().split())))[:z]
                Population_data = final_population
                print("Final poems")
                display(final_population)

            else:
                print("###################################################")
                print("Last Poems Standing")
                print("###################################################")
                display(final_population)

In [10]:
#4 (words_df,intial_population,crossover,epochs,user_selection_poems_number):
poem = poem_generation(words_df,20,2,12,5)


Final poems


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,a own was will,she god lead each,bang i where you,it spare here at,4,4,5,6,10,0.0,"[a, own, was, will, she, god, lead, each, bang, i, where, you, it, spare, here, at]"
1,and united been out,we cork wave years,lead spaces up with,dawn pulled we at,6,4,5,5,13,0.0,"[and, united, been, out, we, cork, wave, years, lead, spaces, up, with, dawn, pulled, we, at]"
2,all so their our,faith adverse cosmos everything,in wronged or how,not wonder go life,4,10,5,5,13,7.397612420591253e-05,"[all, so, their, our, faith, adverse, cosmos, everything, in, wronged, or, how, not, wonder, go, life]"
3,will refrain they love,is hear this not,inside say lines their,the stained own created,5,4,7,6,15,1.695286179718829e-05,"[will, refrain, they, love, is, hear, this, not, inside, say, lines, their, the, stained, own, created]"
4,is here a in,circle knocking there a,its short cosmos lifter,reappearing life only dissolves,5,7,6,9,22,7.705846271449222e-06,"[is, here, a, in, circle, knocking, there, a, its, short, cosmos, lifter, reappearing, life, only, dissolves]"
5,within with positive dreams,in the cry creatures,anchor life matter pattern,it swallow gone spending,8,6,7,6,16,2.844157805643986e-05,"[within, with, positive, dreams, in, the, cry, creatures, anchor, life, matter, pattern, it, swallow, gone, spending]"
6,let songs their spring,love aware can it,needs disappear edge just,walk nothing life fireflylike,4,6,7,9,16,0.00016868798019681572,"[let, songs, their, spring, love, aware, can, it, needs, disappear, edge, just, walk, nothing, life, fireflylike]"
7,went correctness in despair,every say listened trains,rusty sequin me more,untamed poverty not a,7,8,6,8,21,0.0,"[went, correctness, in, despair, every, say, listened, trains, rusty, sequin, me, more, untamed, poverty, not, a]"
8,not one been to,without attack us arm,seeing hand undertow bed,of how everything nightgown,5,6,6,8,18,3.236455434008674e-06,"[not, one, been, to, without, attack, us, arm, seeing, hand, undertow, bed, of, how, everything, nightgown]"
9,pay in a own,was will she god,lead each bang i,where you it spare,4,4,4,6,15,0.0,"[pay, in, a, own, was, will, she, god, lead, each, bang, i, where, you, it, spare]"



Final poems


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,will refrain they love,is hear this not,inside say lines their,the stained own created,5,4,7,6,15,1.695286179718829e-05,"[will, refrain, they, love, is, hear, this, not, inside, say, lines, their, the, stained, own, created]"
1,not one been to,without attack us arm,seeing hand undertow bed,of how everything nightgown,5,6,6,8,18,3.236455434008674e-06,"[not, one, been, to, without, attack, us, arm, seeing, hand, undertow, bed, of, how, everything, nightgown]"
2,pay in a own,was will she god,lead each bang i,where you it spare,4,4,4,6,15,0.0,"[pay, in, a, own, was, will, she, god, lead, each, bang, i, where, you, it, spare]"
3,faith adverse cosmos everything,in wronged or how,not all so will,refrain they love is,10,5,4,5,17,2.1576369560057823e-05,"[faith, adverse, cosmos, everything, in, wronged, or, how, not, all, so, will, refrain, they, love, is]"
4,let songs love aware,can it needs disappear,edge just walk nothing,life fireflylike gone spending,6,6,6,9,14,0.00016451981789544092,"[let, songs, love, aware, can, it, needs, disappear, edge, just, walk, nothing, life, fireflylike, gone, spending]"
5,is reason it man,opens at excesses time,me all inside one,that our fear through,5,7,7,4,13,4.4693908374405494e-05,"[is, reason, it, man, opens, at, excesses, time, me, all, inside, one, that, our, fear, through]"
6,then horse alone bang,live panic very us,she back an that,the hearts is humankind,7,6,4,6,14,5.394092390014456e-06,"[then, horse, alone, bang, live, panic, very, us, she, back, an, that, the, hearts, is, humankind]"
7,carpet forever efforts unrealities,went correctness in a,observed ours without belongs,are of but insane,11,6,8,7,21,0.0,"[carpet, forever, efforts, unrealities, went, correctness, in, a, observed, ours, without, belongs, are, of, but, insane]"
8,light despair every say,listened trains rusty sequin,me more untamed poverty,not not the when,7,8,8,4,18,4.79474879112396e-06,"[light, despair, every, say, listened, trains, rusty, sequin, me, more, untamed, poverty, not, not, the, when]"
9,infused now flame to,now informed grief a,own was will she,god lead each bang,7,6,4,4,16,0.0,"[infused, now, flame, to, now, informed, grief, a, own, was, will, she, god, lead, each, bang]"



Final poems


...

,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,out pay in a,own let songs love,aware aware the hearts,is humankind edge this,4,4,8,7,13,3.7758646730101195e-05,"[out, pay, in, a, own, let, songs, love, aware, aware, the, hearts, is, humankind, edge, this]"
1,edge just walk was,coming given own let,songs their spring love,aware can it needs,5,6,4,6,20,2.8768492746743764e-05,"[edge, just, walk, was, coming, given, own, let, songs, their, spring, love, aware, can, it, needs]"
2,she a own let,songs love aware aware,the hearts is out,pay in humankind edge,4,8,4,7,14,3.7758646730101195e-05,"[she, a, own, let, songs, love, aware, aware, the, hearts, is, out, pay, in, humankind, edge]"
3,can without aware the,can stained an that,to without not all,so our it own,7,5,5,4,10,0.0,"[can, without, aware, the, can, stained, an, that, to, without, not, all, so, our, it, own]"
4,let songs love aware,the hearts is humankind,edge you each bang,i where you love,6,6,5,5,11,8.360843204522408e-05,"[let, songs, love, aware, the, hearts, is, humankind, edge, you, each, bang, i, where, you, love]"
5,lead each bang i,where aware the hearts,is you it love,aware own was will,4,7,4,6,16,1.0788184780028913e-05,"[lead, each, bang, i, where, aware, the, hearts, is, you, it, love, aware, own, was, will]"
6,aware aware the hearts,in humankind edge can,it needs edge just,walk was coming they,8,7,5,5,13,0.0,"[aware, aware, the, hearts, in, humankind, edge, can, it, needs, edge, just, walk, was, coming, they]"
7,disappear edge love aware,aware the hearts is,it needs it lead,each love this it,9,6,4,4,20,1.348523097503614e-05,"[disappear, edge, love, aware, aware, the, hearts, is, it, needs, it, lead, each, love, this, it]"
8,can it needs disappear,not all so will,edge out pay in,refrain they love is,6,4,5,5,17,1.941873260405204e-05,"[can, it, needs, disappear, not, all, so, will, edge, out, pay, in, refrain, they, love, is]"
9,edge just walk was,coming given own let,songs their spring love,can it needs can,5,6,4,4,18,3.2364554340086736e-05,"[edge, just, walk, was, coming, given, own, let, songs, their, spring, love, can, it, needs, can]"


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,edge just walk was,coming given own let,songs their spring love,aware can it needs,5,6,4,6,20,2.8768492746743764e-05,"[edge, just, walk, was, coming, given, own, let, songs, their, spring, love, aware, can, it, needs]"
1,she a own let,songs love aware aware,the hearts is out,pay in humankind edge,4,8,4,7,14,3.7758646730101195e-05,"[she, a, own, let, songs, love, aware, aware, the, hearts, is, out, pay, in, humankind, edge]"
2,can without aware the,can stained an that,to without not all,so our it own,7,5,5,4,10,0.0,"[can, without, aware, the, can, stained, an, that, to, without, not, all, so, our, it, own]"
3,let songs love aware,the hearts is humankind,edge you each bang,i where you love,6,6,5,5,11,8.360843204522408e-05,"[let, songs, love, aware, the, hearts, is, humankind, edge, you, each, bang, i, where, you, love]"
4,lead each bang i,where aware the hearts,is you it love,aware own was will,4,7,4,6,16,1.0788184780028913e-05,"[lead, each, bang, i, where, aware, the, hearts, is, you, it, love, aware, own, was, will]"
5,bang i where you,not songs love aware,the hearts is humankind,edge you each bang,5,6,6,5,14,2.7741046577217202e-05,"[bang, i, where, you, not, songs, love, aware, the, hearts, is, humankind, edge, you, each, bang]"


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
3,can without aware the,can stained an that,to without not all,so our it own,7,5,5,4,10,0.0,"[can, without, aware, the, can, stained, an, that, to, without, not, all, so, our, it, own]"
13,all so our it,own let songs love,i where you love,let edge you each,4,4,5,5,10,0.00020713314777655512,"[all, so, our, it, own, let, songs, love, i, where, you, love, let, edge, you, each]"
18,it needs in humankind,edge disappear walk bang,i refrain they love,she a own let,6,7,5,4,10,5.79864931926554e-05,"[it, needs, in, humankind, edge, disappear, walk, bang, i, refrain, they, love, she, a, own, let]"
10,a own let their,that edge just walk,was coming they love,is faith all so,4,5,5,4,10,5.178328694413878e-05,"[a, own, let, their, that, edge, just, walk, was, coming, they, love, is, faith, all, so]"
4,let songs love aware,the hearts is humankind,edge you each bang,i where you love,6,6,5,5,11,8.360843204522408e-05,"[let, songs, love, aware, the, hearts, is, humankind, edge, you, each, bang, i, where, you, love]"
16,let a it own,let songs their spending,bang just walk just,so will pay in,4,5,4,4,12,1.4384246373371882e-05,"[let, a, it, own, let, songs, their, spending, bang, just, walk, just, so, will, pay, in]"
12,all so our it,bang i where you,not all so our,it edge you each,4,5,4,5,12,0.0,"[all, so, our, it, bang, i, where, you, not, all, so, our, it, edge, you, each]"
0,out pay in a,own let songs love,aware aware the hearts,is humankind edge this,4,4,8,7,13,3.7758646730101195e-05,"[out, pay, in, a, own, let, songs, love, aware, aware, the, hearts, is, humankind, edge, this]"
6,aware aware the hearts,in humankind edge can,it needs edge just,walk was coming they,8,7,5,5,13,0.0,"[aware, aware, the, hearts, in, humankind, edge, can, it, needs, edge, just, walk, was, coming, they]"
2,she a own let,songs love aware aware,the hearts is out,pay in humankind edge,4,8,4,7,14,3.7758646730101195e-05,"[she, a, own, let, songs, love, aware, aware, the, hearts, is, out, pay, in, humankind, edge]"



###################################################
Last Poems Standing
###################################################


,Line1,Line2,Line3,Line4,Syllables1,Syllables2,Syllables3,Syllables4,GrammarError,Coherence,words_array
0,out pay in a,own let songs love,aware aware the hearts,is humankind edge this,4,4,8,7,13,3.7758646730101195e-05,"[out, pay, in, a, own, let, songs, love, aware, aware, the, hearts, is, humankind, edge, this]"
1,edge just walk was,coming given own let,songs their spring love,aware can it needs,5,6,4,6,20,2.8768492746743764e-05,"[edge, just, walk, was, coming, given, own, let, songs, their, spring, love, aware, can, it, needs]"
2,can without aware the,can stained an that,to without not all,so our it own,7,5,5,4,10,0.0,"[can, without, aware, the, can, stained, an, that, to, without, not, all, so, our, it, own]"
3,let songs love aware,the hearts is humankind,edge you each bang,i where you love,6,6,5,5,11,8.360843204522408e-05,"[let, songs, love, aware, the, hearts, is, humankind, edge, you, each, bang, i, where, you, love]"
4,disappear edge love aware,aware the hearts is,it needs it lead,each love this it,9,6,4,4,20,1.348523097503614e-05,"[disappear, edge, love, aware, aware, the, hearts, is, it, needs, it, lead, each, love, this, it]"
5,edge just walk was,coming given own let,songs their spring love,can it needs can,5,6,4,4,18,3.2364554340086736e-05,"[edge, just, walk, was, coming, given, own, let, songs, their, spring, love, can, it, needs, can]"
6,can without own let,songs love own aware,the can stained an,that to without not,5,6,5,5,15,4.3152739120115646e-05,"[can, without, own, let, songs, love, own, aware, the, can, stained, an, that, to, without, not]"
7,all so our it,bang i where you,not all so our,it edge you each,4,5,4,5,12,0.0,"[all, so, our, it, bang, i, where, you, not, all, so, our, it, edge, you, each]"
8,all so our it,own let songs love,i where you love,let edge you each,4,4,5,5,10,0.00020713314777655512,"[all, so, our, it, own, let, songs, love, i, where, you, love, let, edge, you, each]"
9,edge just walk was,coming given own love,aware can it needs,let songs their spring,5,6,6,4,18,2.8768492746743764e-05,"[edge, just, walk, was, coming, given, own, love, aware, can, it, needs, let, songs, their, spring]"


In [11]:
print(list(range(5,8)))

[5, 6, 7]


In [12]:
a = [2,3,4,5]

for i in a:
    print(i)

2
3
4
5
